<div style="padding:20px;color:#DEB078;margin:0;font-size:300%;text-align:center;display:fill;border-radius:5px;background-color:#1F4F59;overflow:hidden;font-weight:800"> Dashboard  </div>

## <b><span style='color:#DEB078'>1 |</span><span style='color:#1F4F59'> Edit Your username</span></b>  

In [ ]:
Enter_Your_Username='abhi011097' # Enter_Your_Username='xyz'

## <b><span style='color:#DEB078'>2 |</span><span style='color:#1F4F59'> Import Library</span></b>  

In [ ]:
from IPython.display import clear_output
!pip install pyspark
clear_output()
from pyspark.sql import SparkSession
import numpy as np 
import pandas as pd 
import warnings
from pyspark.sql import functions as F
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
spark = SparkSession.builder \
          .master("local[*]") \
          .appName("Data Analysis") \
          .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
clear_output()

spark_df_users=spark.read.options(header="true",inferSchema='True').csv('/kaggle/input/meta-kaggle/Users.csv')
spark_df_kernelvotes=spark.read.options(header="true",inferSchema='True').csv('/kaggle/input/meta-kaggle/KernelVotes.csv')
spark_df_kernels=spark.read.options(header="true",inferSchema='True').csv('/kaggle/input/meta-kaggle/Kernels.csv')
spark_df_kernelversions=spark.read.options(header="true",inferSchema='True').csv('/kaggle/input/meta-kaggle/KernelVersions.csv')

spark_df_users.createOrReplaceTempView("users")
spark_df_kernels.createOrReplaceTempView("kernels")
spark_df_kernelvotes.createOrReplaceTempView("kernelvotes")
spark_df_kernelversions.createOrReplaceTempView("kernelversions")

result = spark.sql("select * from users where upper(UserName)=upper('"+Enter_Your_Username+"')" ).toPandas()
user_kaggle_key=result['Id'].values[0]

## <b><span style='color:#DEB078'>3 |</span><span style='color:#1F4F59'> Number of Notebooks</span></b>  

In [ ]:
query="select Count(1) No_Of_Public_Kernels from kernels where AuthorUserId="+str(user_kaggle_key)
spark.sql(query).show()

In [ ]:
query="select CurrentUrlSlug as Top_5_Kernel_Name,TotalComments from kernels where AuthorUserId="+str(user_kaggle_key)+" order by TotalComments desc limit 5"
TotalComments = spark.sql(query).toPandas()
query="select CurrentUrlSlug as Top_5_Kernel_Name,TotalViews  from kernels where AuthorUserId="+str(user_kaggle_key)+" order by TotalViews desc limit 5"
TotalViews = spark.sql(query).toPandas()
query="select CurrentUrlSlug as Top_5_Kernel_Name,TotalVotes from kernels where AuthorUserId="+str(user_kaggle_key)+" order by TotalVotes desc limit 5"
TotalVotes = spark.sql(query).toPandas()

fig = make_subplots(rows=3, cols=1, specs=[[{"type": "table"}], [{"type": "table"}],[{"type": "table"}]],vertical_spacing = 0.05,
                    subplot_titles=('Based on Comments','Based on Views','Based on Votes'))

fig.add_trace(
    go.Table(
        header=dict(values=list(TotalComments.columns),
                align='center'),
        cells=dict(values=TotalComments.transpose().values.tolist(),
               align='center'),
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Table(
        header=dict(values=list(TotalViews.columns),
                align='center'),
        cells=dict(values=TotalViews.transpose().values.tolist(),
               align='center'),
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Table(
        header=dict(values=list(TotalVotes.columns),
                align='center'),
        cells=dict(values=TotalVotes.transpose().values.tolist(),
               align='center'),
    ),
    row=3,
    col=1,
)
fig.update_traces(cells_font=dict(size = 11))
fig.update_layout(
    margin=dict(l=0 ,t=40),height=600
)
fig.show()

## <b><span style='color:#DEB078'>4 |</span><span style='color:#1F4F59'> Distribution of Votes, Comments and Views Per Kernel</span></b>  

In [ ]:
query="""
select views,percentile_views,comments,percentile_comments,votes,percentile_votes from
(select uid,views, 
concat(round((percent_rank(views) OVER (Order BY views))*100,3),' %') as percentile_views,
comments,
concat(round((percent_rank(comments) OVER (Order BY comments))*100,3),' %') as percentile_comments,
votes,
concat(round((percent_rank(votes) OVER (Order BY votes))*100,3),' %') as percentile_votes

        from (
        select AuthorUserId uid, 
        sum(TotalComments) comments,
        sum(TotalViews) views,
        sum(TotalVotes) votes 
        from kernels
        group by 1) )
where uid="""+str(user_kaggle_key)

result = spark.sql(query).show()

In [ ]:
query="select TotalComments,TotalViews,TotalVotes from kernels where AuthorUserId="+str(user_kaggle_key)
result = spark.sql(query).toPandas()
vars = ['TotalComments','TotalViews','TotalVotes']
fig = make_subplots(rows=1, cols=len(vars), subplot_titles=['Comments Distribution','Views Distribution','Votes Distribution'])
for i, var in enumerate(vars):
    fig.add_trace(
        go.Box(y=result[var],
        name=var),
        row=1, col=i+1
    )

fig.update_traces(boxpoints='all', jitter=.3)
fig.update_layout(showlegend=False)
fig.show()

## <b><span style='color:#DEB078'>5 |</span><span style='color:#1F4F59'> Upvoters Category Distribution</span></b>  

In [ ]:
query= """select case when PerformanceTier =0  then 'Novice'
                      when PerformanceTier =1  then 'Contributor'
                      when PerformanceTier =2  then 'Expert'
                      when PerformanceTier =3  then 'Master'
                      when PerformanceTier =4  then 'GrandMaster'
                      end PerformanceTier
             ,count(1) votes
             from users t1 inner join  
                ( select UserId 
                   from kernelvotes 
                    where KernelVersionId in (select distinct Id from kernelversions where AuthorUserId="""+str(user_kaggle_key)+"""))
        t2
        on t1.Id=t2.UserId
        Group by 1 
        order by 2 desc
        """


result = spark.sql(query).toPandas()

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "table"}, {'type':'domain'}]],horizontal_spacing = 0.005,
                        subplot_titles=('Voters Category Classification',''))

fig.add_trace(
    go.Table(
        header=dict(values=list(result.columns),
                align='center'),
        cells=dict(values=result.transpose().values.tolist(),
               align='center'),
    ),
    row=1,
    col=1
)

fig.add_trace(
        go.Pie(labels=result["PerformanceTier"],
               values=result["votes"],
               showlegend=True,
               textposition='inside',
              ),
    row=1,
    col=2,  
)


fig.show()

## <b><span style='color:#DEB078'>6 |</span><span style='color:#1F4F59'> Top Upvoters since 2016</span></b>  

In [ ]:
query="""select t1.DisplayName VoterName, t2.No_Of_Upvotes from users t1 inner join  
        ( select UserId ,Count(1) No_Of_Upvotes 
           from kernelvotes 
            where KernelVersionId in (select distinct Id from kernelversions 
                                        where AuthorUserId=(select min(Id) from users where upper(UserName)=upper('"""+Enter_Your_Username+"""')))
           group by 1 order by 2 desc
         )
        t2
        on t1.Id=t2.UserId
        order by 2 desc 
        limit 15
        """
result = spark.sql(query).show()

## <b><span style='color:#DEB078'>7 |</span><span style='color:#1F4F59'> Last 30 Votes</span></b>  

In [ ]:
query="""select t2.Title,t1.VoteDate,t2.ScriptId, t3.UserName,case when t3.PerformanceTier =0  then 'Novice'
                      when t3.PerformanceTier =1  then 'Contributor'
                      when t3.PerformanceTier =2  then 'Expert'
                      when t3.PerformanceTier =3  then 'Master'
                      when t3.PerformanceTier =4  then 'GrandMaster'
                      end PerformanceTier 
                      , No_Of_Upvotes as Total_Votes_By_User
                      from
        (select * from kernelvotes 
       where KernelVersionId in (select distinct Id from kernelversions where AuthorUserId="""+str(user_kaggle_key)+""")) t1
        inner join 
        (select * from kernelversions where AuthorUserId="""+str(user_kaggle_key)+""") t2        
        on t1.KernelVersionId=t2.id
        inner join 
        users t3
        on t1.UserId=t3.Id
        inner join
         (select UserId ,Count(1) No_Of_Upvotes 
           from kernelvotes 
            where KernelVersionId in (select distinct Id from kernelversions where AuthorUserId="""+str(user_kaggle_key)+""") 
           group by 1 ) t4
        on t1.UserId=t4.UserId
        
        order by  to_Date(t1.VoteDate,'MM/dd/yyyy') desc,t1.id desc
        
"""
vots=spark.sql(query).toPandas()
vots.loc[:30,['Title','VoteDate','UserName','PerformanceTier','Total_Votes_By_User']].style.hide_index()